# Бенчмарк для задачи OneTwoTrip Contest

https://boosters.pro/championship/onetwotrip_challenge/overview
    
    
для студентов ВМК МГУ

2019, Александр Дьяконов www.dyakonov.org/ag/

In [1]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np
# для встроенных картинок
%pylab inline
# отключить предупреждения
import warnings
warnings.filterwarnings('ignore')
# прогресс-бар
from tqdm import tqdm, tqdm_notebook

Populating the interactive namespace from numpy and matplotlib


## загрузка данных

In [2]:
data_train = pd.read_csv('../data/onetwotrip_challenge_train.csv')
data_test = pd.read_csv('../data/onetwotrip_challenge_test.csv')
print ('размеры:', data_train.shape, data_test.shape)

размеры: (196056, 43) (455011, 37)


## посмотрим на данные

обратите внимание, как выводятся дата-фреймы

In [3]:
data_train.head()

,orderid,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25,goal21,goal22,goal23,goal24,goal25,goal1
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,...,1,0,1,1,0,1,0,0,0,0
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,...,1,0,1,0,0,0,0,0,0,0
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,...,1,0,1,1,0,0,0,0,0,0
3,3,0392247b4b87674aba2c32bf2292b105771a6a376871be...,0,-0.238651,10,11,1,1,3,2,...,1,0,1,1,0,0,0,0,0,0
4,4,d1aeefef311bbeb4bd84876c8d49421f276674527d5578...,0,-0.704079,8,11,1,1,0,1,...,1,0,0,1,0,0,0,0,0,0


In [4]:
data_test.head()

,orderid,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,field25,field26,field27,field28,field29,indicator_goal21,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25
0,0,3a6a6af55e097d3f92705936a7ea3ca8aef651f5966832...,0,-0.548937,10,10,1,1,0,1,...,1,1,1,0,4,1,1,0,0,1
1,1,2df10f61960163da94a4294773ed9c865296e37c330304...,82,-0.626508,3,4,3,0,0,1,...,1,1,1,0,1,1,1,0,1,1
2,2,20dc3fec5b5eb42fbfe08119063c3a0010a73c7ec94abb...,0,-0.548937,6,8,1,1,0,2,...,35,1,1,0,2,1,0,0,1,1
3,3,ed75b3496977bac207eccb59dc91fe9a8d6a27777a6422...,6,0.304348,7,7,2,0,10,1,...,1,3,1,0,3,1,1,0,1,1
4,4,a346d08351c5fd0bda82984ed7c8b12b6395829da5b857...,115,-0.471365,3,3,2,0,0,1,...,1,1,1,0,1,1,1,0,1,1


### Проведем немного FE (feature enginering):

In [5]:
full_data = pd.concat([data_train, data_test], axis=0, sort=False)

In [6]:
full_data = full_data.fillna(-1)

In [7]:
full_data['tickets_amount_wout_baby'] = full_data['field15'] - full_data['field9']
full_data['order_cost_1'] = full_data['field14'] * full_data['field15']
full_data['prices_ratio'] = full_data['field1'] / (full_data['field14'] * full_data['field15'])

In [8]:
# full_data = pd.merge(full_data.reset_index(), full_data.groupby('userid')['field0'].agg(['count']), on='userid', how='left').set_index('orderid')

In [9]:
full_data = pd.merge(full_data.reset_index(), full_data.groupby('userid')['field1'].agg(['mean','std']), on='userid').rename(columns={'mean':'field1_stat_mean','std':'field1_stat_std'})
full_data = pd.merge(full_data, full_data.groupby('userid')['field14'].agg(['mean','std']), on='userid').rename(columns={'mean':'field14_stat_mean','std':'field14_stat_std'})
full_data = pd.merge(full_data, full_data.groupby('userid')['field24'].agg(['mean','std']), on='userid').rename(columns={'mean':'field24_stat_mean','std':'field24_stat_std'}).set_index('orderid')

In [10]:
data_train = full_data[full_data['goal1'] >= 0]
data_test = full_data[full_data['goal1'] < 0]

## получаем таблички для обучения

In [11]:
y = data_train.pop('goal1') # целевая переменная для первой задачи

In [12]:
y = y.values # мне так спокойней - в numpy.array

In [13]:
# что получилось
data_train.head()

,index,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,goal25,tickets_amount_wout_baby,order_cost_1,prices_ratio,field1_stat_mean,field1_stat_std,field14_stat_mean,field14_stat_std,field24_stat_mean,field24_stat_std
orderid,,,,,,,,,,,,,,,,,,,,,
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,...,0.0,1,-0.661308,0.947377,-0.626508,NaN,-0.661308,NaN,1.0,NaN
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,...,0.0,1,-0.101043,3.897296,-0.393794,0.000000,-0.171076,0.099042,1.0,0.0
174861,174861,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,0,-0.393794,12,2,1,1,0,2,...,0.0,1,-0.241109,1.633260,-0.393794,0.000000,-0.171076,0.099042,1.0,0.0
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,...,0.0,1,-0.661308,0.830077,-0.587722,0.054851,-0.661308,0.000000,1.0,0.0
173820,173820,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,0,-0.626508,10,10,1,1,0,1,...,0.0,1,-0.661308,0.947377,-0.587722,0.054851,-0.661308,0.000000,1.0,0.0


In [14]:
# удаляем ненужные признаки
ids = data_test.pop('index') # сохраняем id для теста


In [15]:
data_test.drop(['userid'], inplace=True, axis=1)
data_test.drop(['goal1'], inplace=True, axis=1)
data_test.drop(['goal21'], inplace=True, axis=1)
data_test.drop(['goal23'], inplace=True, axis=1)
data_test.drop(['goal24'], inplace=True, axis=1)
data_test.drop(['goal25'], inplace=True, axis=1)
data_test.drop(['indicator_goal21'], inplace=True, axis=1)
data_test.drop(['indicator_goal22'], inplace=True, axis=1)
data_test.drop(['indicator_goal23'], inplace=True, axis=1)
data_test.drop(['indicator_goal24'], inplace=True, axis=1)
data_test.drop(['indicator_goal25'], inplace=True, axis=1)

In [16]:
cols = data_test.columns # значимые колонки
cols

Index(['field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6',
       'field7', 'field8', 'field9', 'field10', 'field11', 'field12',
       'field13', 'field14', 'field15', 'field16', 'field17', 'field18',
       'field19', 'field20', 'field21', 'field22', 'field23', 'field24',
       'field25', 'field26', 'field27', 'field28', 'field29', 'goal22',
       'tickets_amount_wout_baby', 'order_cost_1', 'prices_ratio',
       'field1_stat_mean', 'field1_stat_std', 'field14_stat_mean',
       'field14_stat_std', 'field24_stat_mean', 'field24_stat_std'],
      dtype='object')

In [17]:
data_train = data_train[cols]

### Эксперименты

сначала делим выборку на обучение и тест

не самое лучшее решение, но для быстроты экспериментов сгодится

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, y, test_size=0.3, random_state=13)

### Случайный лес


строим по одному дереву и вычисляем метрику качества (ROC AUC)

обратите внимание на прогресс-бар

In [19]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score

# model = RandomForestClassifier(max_features=1, n_estimators=1, oob_score=False, warm_start=True, random_state=1)

# aucs = []
# for t in tqdm_notebook(list(range(1, 101))):
#     model.set_params(n_estimators=t)
#     model.fit(X_train, y_train)
#     a = model.predict_proba(X_test)[:, 1]
#     q = roc_auc_score(y_test, a)
#     aucs.append(q)
    
# plt.plot(range(1, 101), aucs)

In [20]:
# # аналогичные эксперименты с max_features=2

# model = RandomForestClassifier(max_features=2, n_estimators=1, oob_score=False, warm_start=True, random_state=1)

# aucs = []
# for t in tqdm_notebook(list(range(1, 101))):
#     model.set_params(n_estimators=t)
#     model.fit(X_train, y_train)
#     a = model.predict_proba(X_test)[:, 1]
#     q = roc_auc_score(y_test, a)
#     aucs.append(q)
    
# plt.plot(range(1, 101), aucs)   

# Козырь

более хорошая модель из другой библиотеки

In [21]:
import lightgbm as lgb
model = lgb.LGBMClassifier(num_leaves=6, learning_rate=0.08, n_estimators=700)

In [22]:
%%time

param = {'num_leaves': 6, 'objective': 'binary', 'learning_rate': 0.1}
param['metric'] = 'auc'

q = lgb.train(param, train_set=lgb.Dataset(X_train, y_train), num_boost_round=500,
              valid_sets=[lgb.Dataset(X_train, y_train), lgb.Dataset(X_test, y_test)],
             verbose_eval=50)

[50]	valid_0's auc: 0.705992	valid_1's auc: 0.688063
[100]	valid_0's auc: 0.727535	valid_1's auc: 0.688323
[150]	valid_0's auc: 0.7424	valid_1's auc: 0.690084
[200]	valid_0's auc: 0.754498	valid_1's auc: 0.691827
[250]	valid_0's auc: 0.765674	valid_1's auc: 0.693942
[300]	valid_0's auc: 0.772617	valid_1's auc: 0.693574
[350]	valid_0's auc: 0.781061	valid_1's auc: 0.694724
[400]	valid_0's auc: 0.789271	valid_1's auc: 0.696108
[450]	valid_0's auc: 0.797197	valid_1's auc: 0.696654
[500]	valid_0's auc: 0.803738	valid_1's auc: 0.695009
CPU times: user 59.7 s, sys: 1min 20s, total: 2min 20s
Wall time: 4.49 s


[50]	valid_0's auc: 0.709098	valid_1's auc: 0.675885
[100]	valid_0's auc: 0.726553	valid_1's auc: 0.679719
[150]	valid_0's auc: 0.736703	valid_1's auc: 0.679034
[200]	valid_0's auc: 0.747241	valid_1's auc: 0.678564
[250]	valid_0's auc: 0.756099	valid_1's auc: 0.67807
[300]	valid_0's auc: 0.762981	valid_1's auc: 0.677406
[350]	valid_0's auc: 0.771099	valid_1's auc: 0.676982
[400]	valid_0's auc: 0.778818	valid_1's auc: 0.676227
[450]	valid_0's auc: 0.783759	valid_1's auc: 0.675318
[500]	valid_0's auc: 0.789803	valid_1's auc: 0.67336
CPU times: user 2min 20s, sys: 3.32 s, total: 2min 23s

Воспользуемся bayes_opt для подбора параметров:

In [23]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score


def lgb_bayesian(max_depth,
                 lambda_l1,
                 lambda_l2,
                 bagging_fraction,
                 bagging_freq,
                 colsample_bytree,
                 learning_rate,
                 num_leaves,
                 max_bin
                      ):
    params = {
        'boosting_type': 'gbdt',
        'metric': 'AUC',
        'objective': 'binary',
        'eval_metric': 'AUC',
        'n_jobs': 16,
        'seed': 42,
        'early_stopping_rounds': 150,
        'n_estimators': 2000,
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'verbose': 0
    }
    model = lgb.LGBMClassifier(boosting_type=params['boosting_type'],
                                metric='AUC',
                                objective='binary',
                                eval_metric='AUC',
                                n_jobs= 16,
                                seed=42,
#                                 early_stopping_rounds=150,
                                n_estimators= 2000,
                                learning_rate=learning_rate,
                                max_depth=int(max_depth),
                                lambda_l1=lambda_l1,
                                lambda_l2=lambda_l2,
                                bagging_fraction=bagging_fraction,
                                bagging_freq=int(bagging_freq),
                                max_bin=int(max_bin),
                                num_leaves=int(num_leaves),
                                colsample_bytree=colsample_bytree,
                                verbose=0,
#                                 valid_sets=[lgb.Dataset(X_train, y_train), lgb.Dataset(X_test, y_test)]
                                )
    model.fit(X_train, y_train)

    return roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [24]:
bounds_LGB = {
    'max_depth': (2, 20),
    'lambda_l1': (0, 10),
    'lambda_l2': (0, 10),
    'bagging_fraction': (0.4, 0.9),
    'bagging_freq': (1, 10),
    'colsample_bytree': (0.4, 0.9),
    'learning_rate': (0.025, 0.1),
    'num_leaves': (2, 20),
    'max_bin': (2, 1000)
}


lgbm_BO = BayesianOptimization(lgb_bayesian, bounds_LGB, random_state=13 * 2)

In [25]:
%%time

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    lgbm_BO.maximize(init_points=20, n_iter=20)

|   iter    |  target   | baggin... | baggin... | colsam... | lambda_l1 | lambda_l2 | learni... |  max_bin  | max_depth | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6898   |  0.554    |  5.675    |  0.7841   |  7.892    |  8.706    |  0.03909  |  271.0    |  10.93    |  15.3     |
|  2        |  0.6786   |  0.4975   |  2.618    |  0.6694   |  2.828    |  7.609    |  0.04523  |  294.9    |  19.32    |  18.44    |
|  3        |  0.6926   |  0.6841   |  5.369    |  0.7269   |  5.366    |  9.957    |  0.03565  |  975.4    |  13.15    |  14.97    |
|  4        |  0.675    |  0.5475   |  6.401    |  0.7753   |  6.414    |  9.233    |  0.09165  |  645.4    |  7.322    |  14.29    |
|  5        |  0.6628   |  0.5385   |  2.138    |  0.685    |  4.091    |  5.604    |  0.08908  |  98.53    |  8.378    |  19.53    |
|  6        |  0.6926   |  0.861    |  9.718    |  0.4255   | 

|  37       |  0.689    |  0.8267   |  9.751    |  0.5156   |  0.2577   |  8.476    |  0.02651  |  669.3    |  19.53    |  2.295    |
|  38       |  0.6885   |  0.4468   |  9.05     |  0.6495   |  1.882    |  8.643    |  0.03118  |  101.0    |  2.894    |  2.615    |
|  39       |  0.6981   |  0.5668   |  9.697    |  0.8251   |  8.594    |  9.904    |  0.04736  |  171.6    |  2.676    |  4.225    |
|  40       |  0.699    |  0.8366   |  8.498    |  0.4395   |  0.6659   |  9.661    |  0.05012  |  949.3    |  2.04     |  3.056    |
CPU times: user 48min 43s, sys: 30.1 s, total: 49min 14s
Wall time: 4min 46s


In [26]:
## 7017

In [27]:
# model_params = {
#     'max_depth': int(lgbm_BO.max['params']['max_depth']),
# #     'n_estimators': int(lgbm_BO.max['params']['n_estimators']),
#     'learning_rate': lgbm_BO.max['params']['learning_rate'],
#     'num_leaves': int(lgbm_BO.max['params']['num_leaves']),
#     'max_bin': int(lgbm_BO.max['params']['max_bin']),
#     'boosting_type': 'gbdt',
#     'metric': 'AUC',
#     'objective': 'binary',
#     'eval_metric': 'AUC',
#     'n_jobs': 16,
#     'seed': 42,
# #     'early_stopping_rounds': 150,
#     'n_estimators': 2000,
#     'lambda_l1': lambda_l1,
#     'lambda_l2': lambda_l2,
#     'bagging_fraction': lgbm_BO.max['params']['bagging_fraction'],
#     'bagging_freq': int(lgbm_BO.max['params']['bagging_freq']),
#     'colsample_bytree': lgbm_BO.max['params']['colsampel_bytree'],
#     'verbose': 0
# }

In [29]:
model = lgb.LGBMClassifier(
            max_depth=int(lgbm_BO.max['params']['max_depth']),
            learning_rate=lgbm_BO.max['params']['learning_rate'],
            num_leaves=int(lgbm_BO.max['params']['num_leaves']),
            max_bin=int(lgbm_BO.max['params']['max_bin']),
            boosting_type='gbdt',
            metric='AUC',
            objective='binary',
            eval_metric='AUC',
            n_jobs=16,
            seed=42,
        #     'early_stopping_rounds': 150,
            n_estimators=2000,
            lambda_l1=lgbm_BO.max['params']['lambda_l1'],
            lambda_l2=lgbm_BO.max['params']['lambda_l2'],
            bagging_fraction=lgbm_BO.max['params']['bagging_fraction'],
            bagging_freq=int(lgbm_BO.max['params']['bagging_freq']),
            colsample_bytree=lgbm_BO.max['params']['colsample_bytree'],
            verbose=0
        )

In [30]:
model.fit(data_train, y)

LGBMClassifier(bagging_fraction=0.8169284727282202, bagging_freq=5,
               boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.40252215258566176, eval_metric='AUC',
               importance_type='split', lambda_l1=3.6468221495638997,
               lambda_l2=7.69464010412219, learning_rate=0.04567205594602278,
               max_bin=293, max_depth=3, metric='AUC', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
               n_jobs=16, num_leaves=5, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, seed=42, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=0)

In [54]:
%%time

from xgboost import XGBClassifier

xgb_model = XGBClassifier(gpu_hist=True)

xgb_model.fit(X_train, y_train)

CPU times: user 9.32 s, sys: 27.9 ms, total: 9.34 s
Wall time: 9.35 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_hist=True,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [55]:
roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:, 1])

0.6924488241620788

In [57]:
a = xgb_model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [58]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('xgb.csv')

In [33]:
from catboost import CatBoostClassifier


def catboost_bayesian(max_depth,
                      n_estimators,
                      lr,
                      ):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(max_depth=max_depth, n_estimators=n_estimators,
                               learning_rate=lr, objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)
    model.fit(X_train, y_train)

    return roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [34]:
bounds_cat = {
    'max_depth': (1, 16),
    'n_estimators': (1, 1000),
    'lr': (1e-3, 1),
}

cat_BO = BayesianOptimization(catboost_bayesian, bounds_cat, random_state=13 * 2)

In [35]:
%%time
import warnings


with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    cat_BO.maximize(init_points=20, n_iter=20, acq='ucb')

|   iter    |  target   |    lr     | max_depth | n_esti... |
-------------------------------------------------------------
|  1        |  0.581    |  0.3086   |  8.791    |  768.5    |
|  2        |  0.5944   |  0.7894   |  14.06    |  188.7    |
|  3        |  0.5893   |  0.2702   |  8.443    |  739.4    |
|  4        |  0.6869   |  0.1958   |  3.696    |  539.3    |
|  5        |  0.601    |  0.2835   |  12.41    |  270.4    |


KeyboardInterrupt: 

In [36]:
model = CatBoostClassifier(max_depth=int(cat_BO.max['params']['max_depth']),
                           n_estimators=int(cat_BO.max['params']['n_estimators']),
                           learning_rate=cat_BO.max['params']['lr'],
                           objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)

In [37]:
model.fit(data_train, y)

KeyboardInterrupt: 

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('ctb.csv')

### Обучение и формирование ответа

In [ ]:
model.fit(data_train, y)

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('dj1_01_.csv')